In [2]:
import paho.mqtt.publish as publish
from core.settings import settings

In [3]:
publish.single(topic='/mpc',
               hostname=settings.MQTT_HOST,
               port=settings.MQTT_PORT)